In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import  Select
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
# import numpy as np
import time

position_dict = {'QB': 2, 'RB': 3, 'WR': 4, 'TE': 5, 'D/ST': 7, 'K': 8}
week_dict = {"NFL Week 1": 6, "NFL Week 2": 5, "NFL Week 3": 4, "NFL Week 4": 3}

driver = webdriver.Chrome()
driver.get("https://fantasy.espn.com/football/leaders")

In [2]:

def extract_table_to_dataframe(header_xpath: str, tBody_xpath: str) -> pd.DataFrame:
    # Extract the column names from the header row
    header_row = driver.find_element(By.XPATH, header_xpath)
    cols = header_row.find_elements(By.TAG_NAME, 'th')
    column_names = [col.text for col in cols]
    df = pd.DataFrame(columns=column_names)
    tbody = driver.find_element(By.XPATH, tBody_xpath)
    rows = tbody.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        temp_list = [col.text for col in row.find_elements(By.TAG_NAME, 'td')]
        df.loc[len(df)] = temp_list
    return df

In [3]:

final_df = pd.DataFrame()

time.sleep(4)
for week, week_index in week_dict.items():
    dropdown = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[2]/div[1]/div/div[2]/div[5]/div/div[2]/select/option[{week_index}]')
    print(f'Week: {week}')
    dropdown.click()
    time.sleep(3)

    # Loop through positions
    for position, position_index in position_dict.items():
        print(f'Position: {position}')
        label = driver.find_element(By.XPATH, f'//*[@id="filterSlotIds"]/label[{position_index}]')
        label.click()
        
        while True: 
            i = 0
            try: 
                time.sleep(6)
                start = time.time()
                header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[1]/thead/tr[2]'
                tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[1]/tbody'
                df1=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/div/div[2]/table/thead/tr[2]'
                tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/div/div[2]/table/tbody'
                df2=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[2]/thead/tr[2]'
                tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[2]/tbody'
                df3=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                df_combined = pd.concat([df1, df2, df3], axis=1)
                df_combined['week'] = str(week)
                final_df = final_df.append(df_combined, ignore_index=True)

                button = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/nav/button[2]')
                if button.is_enabled():
                    button.click()
                    print(f'Next page {time.time() - start}')
                else:
                    print(f'End of Position {time.time() - start}')
                    break
            except Exception as e:
                print(f'End of Position {time.time() - start}')
                break

Week: NFL Week 1
Position: QB
Next page 14.614105939865112
Next page 13.563340902328491
End of Position 8.512422800064087
Position: RB
Next page 13.914700031280518
Next page 15.0975022315979
Next page 17.364660024642944
End of Position 5.864406108856201
Position: WR
Next page 16.05722689628601
Next page 16.035677194595337
Next page 14.624266862869263
Next page 20.663749933242798
Next page 23.882614135742188
Next page 12.358572006225586
Next page 12.784894943237305
End of Position 10.723572969436646
Position: TE
Next page 13.016056776046753
Next page 15.964860200881958
Next page 14.187206745147705
Next page 13.655429124832153
End of Position 3.8781139850616455
Position: D/ST
End of Position 7.0985658168792725
Position: K
End of Position 9.30216121673584
Week: NFL Week 2
Position: QB
Next page 14.191669702529907
Next page 14.287140130996704
End of Position 9.063352108001709
Position: RB
Next page 14.501384019851685
Next page 15.483839988708496
Next page 14.722670793533325
Next page 14.49

In [9]:
final_df.head()

,PLAYER,TYPE,ACTION,OPP,STATUS,PROJ,C/A,YDS,TD,INT,...,TD,REC,YDS,TD,TAR,2PC,FUML,TD,FPTS,week
0,Tua Tagovailoa\nMia\nQB,,,@LAC,W 36-34,17.6,28/45,466,3,1,...,0,0,0,0,0,0,1,0,27.14,NFL Week 1
1,Mac Jones\nNE\nQB,,,Phi,L 20-25,14.7,35/54,316,3,1,...,0,0,0,0,0,0,0,0,24.14,NFL Week 1
2,Jordan Love\nGB\nQB,,,@Chi,W 38-20,14.8,15/27,245,3,0,...,0,0,0,0,0,0,0,0,23.0,NFL Week 1
3,Anthony Richardson\nInd\nQB,,,Jax,L 21-31,17.2,24/37,223,1,1,...,1,0,0,0,0,0,0,0,20.92,NFL Week 1
4,Justin Herbert\nQ\nLAC\nQB,,,Mia,L 34-36,16.9,23/33,229,1,0,...,1,0,0,0,0,0,0,0,20.86,NFL Week 1


In [ ]:
final_df.to_csv('all_data_wk1-4.csv', index=False) 
final_df.head(200)

In [11]:
# Split the 'Column1' into three separate columns
split_data = final_df['PLAYER'].str.split('\n', expand=True)

# Assign the split columns to the DataFrame
final_df['Name'] = split_data[0]
final_df['Team'] = split_data[1]
final_df['Position'] = split_data[2]


final_df = final_df[['Name', 'Team', 'Position'] + final_df.columns.difference(['Name', 'Team', 'Position']).tolist()]
# Drop the original 'Column1'
final_df = final_df.drop('PLAYER', axis=1)


,TYPE,ACTION,OPP,STATUS,PROJ,C/A,YDS,TD,INT,CAR,...,TD,TAR,2PC,FUML,TD,FPTS,week,Name,Team,Position
0,,,@LAC,W 36-34,17.6,28/45,466,3,1,5,...,0,0,0,1,0,27.14,NFL Week 1,Tua Tagovailoa,Mia,QB
1,,,Phi,L 20-25,14.7,35/54,316,3,1,2,...,0,0,0,0,0,24.14,NFL Week 1,Mac Jones,NE,QB
2,,,@Chi,W 38-20,14.8,15/27,245,3,0,3,...,0,0,0,0,0,23.0,NFL Week 1,Jordan Love,GB,QB
3,,,Jax,L 21-31,17.2,24/37,223,1,1,10,...,0,0,0,0,0,20.92,NFL Week 1,Anthony Richardson,Ind,QB
4,,,Mia,L 34-36,16.9,23/33,229,1,0,5,...,0,0,0,0,0,20.86,NFL Week 1,Justin Herbert,Q,LAC
5,,,Cin,W 24-3,18.5,16/29,154,1,1,5,...,0,0,0,0,0,20.66,NFL Week 1,Deshaun Watson,O,Cle
6,,,Det,L 20-21,24.7,21/39,226,2,1,6,...,0,0,0,0,0,19.54,NFL Week 1,Patrick Mahomes,KC,QB
7,,,@Ind,W 31-21,18.5,24/32,241,2,1,7,...,0,0,0,0,0,17.74,NFL Week 1,Trevor Lawrence,Jax,QB
8,,,@Pit,W 30-7,13.8,19/29,220,2,0,3,...,0,0,0,1,0,16.8,NFL Week 1,Brock Purdy,SF,QB
9,,,TB,L 17-20,17.4,33/44,344,2,1,3,...,0,0,0,2,0,16.46,NFL Week 1,Kirk Cousins,Min,QB


In [13]:
final_df.head()

,Name,Team,Position,2PC,ACTION,C/A,CAR,FPTS,FUML,INT,...,TAR,TD,TD,TD,TD,TYPE,YDS,YDS,YDS,week
0,Tua Tagovailoa,Mia,QB,0,,28/45,5,27.14,1,1,...,0,3,0,0,0,,466,5,0,NFL Week 1
1,Mac Jones,NE,QB,0,,35/54,2,24.14,0,1,...,0,3,0,0,0,,316,15,0,NFL Week 1
2,Jordan Love,GB,QB,0,,15/27,3,23.0,0,0,...,0,3,0,0,0,,245,12,0,NFL Week 1
3,Anthony Richardson,Ind,QB,0,,24/37,10,20.92,0,1,...,0,1,1,0,0,,223,40,0,NFL Week 1
4,Justin Herbert,Q,LAC,0,,23/33,5,20.86,0,0,...,0,1,1,0,0,,229,17,0,NFL Week 1


In [ ]:
df_combined.to_csv('data1-4.csv', index=False, encoding='utf-8-sig')
